In [2]:
from collections import namedtuple
import os
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True) 
class PrepareCallbackConfig:      #this will also work as a named tuple but is a better way to implement the same concept
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoint_model_filepath: Path
    
@dataclass(frozen=True) 
class TrainingConfig:      #this will also work as a named tuple but is a better way to implement the same concept
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [5]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callback
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)   #since it has a filename in the end of filepath
        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_log_dir)])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_log_dir = Path(config.tensorboard_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath))

        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        config_training = self.config.training
        model_config = self.config.prepare_base_model
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"PetImages")
        params = self.params

        create_directories([Path(config_training.root_dir)])

        training_config = TrainingConfig(
            root_dir = Path(config_training.root_dir),
            trained_model_path= Path(config_training.trained_model_path),
            updated_base_model_path= Path(model_config.updated_base_model_path),
            training_data= training_data,
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE)
        return training_config

In [7]:
import time
import tensorflow as tf
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):   #tensorboard callbacks
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")  #good practice to create folders with timestamp names as they are unique
        tb_running_log_dir = os.path.join(self.config.tensorboard_log_dir,(f"tb_logs_at_{timestamp}"))
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):   #checkpoint callbacks    
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_filepath)


    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,   #we dont need to call a property as method i.e. with ()
            self._create_ckpt_callbacks
        ]

2023-04-08 19:37:15.218502: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
import os

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle= False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            training_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip=True,
                width_shift_range=0.2, #20%
                height_shift_range=0.2, #20%
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            training_datagenerator = valid_datagenerator
        
        self.train_generator = training_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path:Path,model:tf.keras.models):
        model.save(path)

    def train(self,callbacks_list:list):
        self.step_per_epoch = self.train_generator.samples/self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples/self.valid_generator.batch_size
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.step_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callbacks_list
        )
        self.save_model(
            self.config.trained_model_path,
            self.model
        )

In [9]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config = prepare_callback_config)
    callback_list = prepare_callback.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callbacks_list=callback_list)
except Exception as e:
    raise e

2023-04-08 19:38:24.303781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 4998 images belonging to 2 classes.
Found 19999 images belonging to 2 classes.
1244/1249 [============================>.] - ETA: 1:51 - loss: 0.6924 - accuracy: 0.5331

/Users/akshayjain/Desktop/Deep Learning Codes/Deep Learning Project/env/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1249/1249 [==============================] - 59617s 48s/step - loss: 0.6924 - accuracy: 0.5332 - val_loss: 0.6924 - val_accuracy: 0.4978


TypeError: save_model() missing 1 required positional argument: 'model'

In [ ]:
ls

LICENCE               logs/                 setup.py
README.md             params.yaml           src/
artifacts/            pyproject.toml        template.py
configs/              requirements.txt      tests/
dvc.yaml              requirements_dev.txt  tox.ini
env/                  research/
init_setup.sh         setup.cfg


In [ ]:
print('hello')

In [10]:
training.model.save('artifacts/training/model.h5')